# Tools & Uitls

## Make this file be able to import other .ipynb files

In [2]:
from IMP import *
from collections import OrderedDict

In [3]:
import os
import sys
from hashlib import md5

class PrintColorfulTraceBack(BaseException):
    ''' This is just to print colorful trace_back and interrupt the function '''

In [4]:
def get_md5_string(string):
    return md5(string.encode()).hexdigest()

In [5]:
def get_addr_port(addr_info):
    linux = sys.platform.lower().startswith('linux')
    try:
        assert('@' in addr_info)        
        addr, port = addr_info.split('@')
        addrlist = addr.split('.')
        alen = len(addrlist)
        assert(alen == 1 or alen == 4)
        
        if alen == 1:
            # this is serial port, address 0 is broadcast channel, do not use it
            a = int(addrlist[0])
            assert(0 < a < 256)
            port = port.lower()
            assert(len(port) > 3)
            if linux:
                assert(port.startswith('tty'))
            else:
                assert(port.startswith('com'))
        else:  
            # this is TCP port
            for x in addrlist:
                assert(x.isnumeric())
                x = int(x)
                assert(0 <= x < 256)
            assert(port.isnumeric())
    except Exception as e:
        print('illegal argument: addr_info')
        print('Usage: 192.168.1.10@8080 or 10@COM6')
        print('NOT %s'%addr_info)
        raise(PrintColorfulTraceBack) # this is ugly, but to print colorful trace_back and interrupt the function
    
    return addr, port

In [6]:
def get_regs_list(regs_list):
    rslt = []
    regs_list = regs_list.replace(' ', '')
    try:
        groups = regs_list.split(',')
        last = -1
        for unit in groups:
            if '-' not in unit:
                # single num
                assert(unit.isnumeric())
                current = int(unit)
                assert(current > last)
                rslt.append(current)
                last = current
            else:
                # like 10-20
                start, end = unit.split('-')
                start = int(start); end = int(end)
                assert(start > last)
                rslt += list(range(start, end+1)) # range(2,5) is [2,3,4]
                last = end
    except Exception as e:
        print('illegal argument: regs_list')
        print('Usage: 10-20,25, 30 - 40')
        print('NOT %s'%regs_list)
        raise(PrintColorfulTraceBack) # this is ugly, but to print colorful trace_back and interrupt the function
    
    return rslt

In [7]:
def get_reg_names(reg_names):
    rslt = map(lambda x: x.strip(), reg_names.split(','))
    return list(rslt)

In [10]:
def sort_dict(d):
    rslt = OrderedDict()
    for k in sorted(d.keys()):
        rslt.update({k: d[k]})
    return rslt

In [9]:
class cached_property(object):
    ''' A property that is only computed once per instance and then replaces
        itself with an ordinary attribute. Deleting the attribute resets the
        property. '''
        
    def __init__(self, func):
        self.__doc__ = func.__doc__
        self.func = func
    
    def __get__(self, obj, cls):
        if obj is None: return self
        value = obj.__dict__[self.func.__name__] = self.func(obj)
        return value